In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import torch
print("当前设备：", torch.cuda.current_device())
print("设备名：", torch.cuda.get_device_name(0))

当前设备： 0
设备名： NVIDIA GeForce RTX 3090


In [3]:
from client import Client
from server import Server
from validation_tools import load_sampled_dataset, compute_prf
from evaluate import load as load_metric
import pandas as pd
from datasets import load_dataset
import validation

# DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

/home/weiwei/miniconda3/envs/rag_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
commonsense = Client(vectorstore_path='./common_sense_db')

/home/weiwei/RAGnet/client.py:62: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=model_path,


In [5]:
commonsense.build_vectorstore(folder_path="./classified/common_sense")

Total chunks after Character split: 1
Total chunks after semantic split: 1
Inserted batch up to merged chunk 1/1
Total chunks after Character split: 1
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after Character split: 1
Total chunks after semantic split: 1
Inserted batch up to merged chunk 1/1
Total chunks after Character split: 1
Total chunks after semantic split: 1
Inserted batch up to merged chunk 1/1
Total chunks after Character split: 1
Total chunks after semantic split: 1
Inserted batch up to merged chunk 1/1
Total chunks after Character split: 1
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after Character split: 1
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after Character split: 1
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after Character split: 1
Total chunks after semantic split: 2
Inserted batch up to 

OutOfMemoryError: CUDA out of memory. Tried to allocate 282.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 280.00 MiB is free. Process 2655498 has 3.75 GiB memory in use. Process 2656900 has 3.74 GiB memory in use. Process 3049749 has 5.06 GiB memory in use. Including non-PyTorch memory, this process has 10.86 GiB memory in use. Of the allocated memory 9.68 GiB is allocated by PyTorch, and 892.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)